In [57]:
import pandas as pd

fh = "../files/picklist/google-candidates-clean.txt"

In [58]:
df = pd.read_csv(
    fh,
    delimiter="\t",
    encoding="utf-8",
    engine="python",
    keep_default_na=False,
    names=[
        "iNumber", "iCode2", "iType", "iLocation", "iStatus", "iMessage", "iOpacMessage",
        "iAgency", "iBarcode", "iUnit", "iCallNum", "iVolume", "iIntNote", "Message",
        "DateType", "Date1", "Date2", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [59]:
print(sorted(df["iLocation"].unique()))

['bur51', 'lsxx2', 'mab82', 'mab88', 'mab92', 'mab98', 'mabb2', 'mabm2', 'mac82', 'macc2', 'mae82', 'maee2', 'maf82', 'maf88', 'maf92', 'maf98', 'maff3', 'mag  ', 'mag82', 'mag92', 'mag98', 'magg1', 'magg2', 'mai32', 'mai82', 'mak32', 'mal  ', 'mal72', 'mal82', 'mal92', 'mall1', 'malm2', 'malv2', 'maor2', 'map  ', 'map82', 'map92', 'map98', 'map99', 'mapp1', 'mapp2', 'mapp3', 'mapp8', 'mapp9', 'marr2', 'mas82', 'mauu2', 'pad22', 'pad32', 'pad38', 'pah32', 'pam11', 'pam32', 'pam38', 'pat11', 'pat28', 'pat32', 'pat38', 'pat42', 'rc2cf', 'rc2ma', 'rccd2', 'rccd9', 'rccf2', 'rcma2', 'rcmb2', 'rcmb8', 'rcmf2', 'rcmf8', 'rcmg2', 'rcmg8', 'rcmg9', 'rcmi2', 'rcml2', 'rcmo2', 'rcmp2', 'rcmr2', 'rcpd2', 'rcph2', 'rcph9', 'rcpm2', 'rcpm8', 'rcpt2', 'rcx28', 'rcxx2', 'sccc2', 'scdd2', 'scff1', 'scff2', 'scff3']


In [60]:
print(sorted(df["iType"].unique()))

[0, 1, 2, 3, 4, 6, 7, 8, 9, 12, 15, 16, 18, 20, 21, 22, 23, 25, 26, 27, 28, 29, 32, 33, 34, 35, 37, 41, 42, 51, 52, 55, 57, 60, 61, 65, 66]


In [61]:
def determine_building(row):
    if row["iLocation"].startswith("rc"):
        return "ReCAP"
    elif row["iLocation"].startswith("ma") and row["iLocation"][3] == "9":
        return "SASB-M2"
    elif row["iLocation"].startswith("ma") and row["iLocation"].endswith("82"):
        return "SASB-M1"
    elif row["iLocation"].startswith("ma"):
        return "SASB"
    elif row["iLocation"].startswith("sc"):
        return "SCH"
    elif row["iLocation"].startswith("pa"):
        return "LPA"
    elif row["iLocation"].startswith("bu"):
        return "SNFL"

In [62]:
def do_not_sent(row):
    if not row["iLocation"].startswith(("bu", "ma", "pa", "rc", "sc")):
        return "TRUE"
    elif int(row["iType"]) > 101:
        return "TRUE"
    elif int(row["iType"]) in (
        4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 26, 27, 28, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 41, 42, 43, 51, 52, 53, 66, 68, 70, 72,
        73, 74, 75, 76, 79, 51, 52, 53):
        return "TRUE"
    elif "google" in row["iIntNote"].lower():
        return "TRUE"
    elif "poor condition" in row["iIntNote"].lower():
        return "TRUE"
    else:
        return "FALSE"
    

In [63]:
df["building"] = df.apply(lambda row: determine_building(row), axis=1)

In [64]:
df["do_not_sent"] = df.apply(lambda row: do_not_sent(row), axis=1)

In [65]:
df_sasb = df[(df["building"] == "SASB") & (df["do_not_sent"] == "FALSE")]
gdf_sasb = df_sasb.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_sasb.to_csv("../files/picklist/sasb.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [66]:
df_m1 = df[(df["building"] == "SASB-M1") & (df["do_not_sent"] == "FALSE")]
gdf_m1 = df_m1.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))  
gdf_m1.to_csv("../files/picklist/sasb-m1.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]) 

In [67]:
df_m2 = df[(df["building"] == "SASB-M2") & (df["do_not_sent"] == "FALSE")]
df_m2.to_csv("../files/picklist/sasb-m2-detailed.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])
df_m2.to_csv("../files/picklist/sasb-m2-barcodes.csv", index=False, sep="\t", columns=["iBarcode"])

In [68]:
df_recap = df[(df["building"] == "ReCAP") & (df["do_not_sent"] == "FALSE")]
df_recap.to_csv("../files/picklist/recap.csv", index=False, sep="\t", columns=["iBarcode"])

In [69]:
df_lpa = df[(df["building"] == "LPA") & (df["do_not_sent"] == "FALSE")]
gdf_lpa = df_lpa.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_lpa.to_csv("../files/picklist/lpa.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [70]:
df_sch = df[(df["building"] == "SCH") & (df["do_not_sent"] == "FALSE")]
gdf_sch = df_sch.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_sch.to_csv("../files/picklist/sch.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [71]:
df_snfl = df[(df["building"] == "SNFL") & (df["do_not_sent"] == "FALSE")]
gdf_snfl = df_snfl.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_snfl.to_csv("../files/picklist/snfl.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [72]:
df_do_not_sent = df[df["do_not_sent"] == "TRUE"]
df_do_not_sent.to_csv("../files/picklist/do_not_sent.csv", index=False, sep="\t", columns=["iBarcode"])